In [2]:
import obj_2_pcd
import render_inference_result
import importlib
#import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import multiprocessing
#from vedo import dataurl, printc, Plotter, Points, Mesh, Text2D
import pytorch3d
import importlib
import obj_2_pcd
import render_inference_result
importlib.reload(obj_2_pcd)
importlib.reload(render_inference_result)
import torch
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


import glob
import os
import shutil

data_ids = ['0408']
tiff_dir_root = '/data/jhahn/data/brain_lightsheet'
obj_dir_root = '/data/jhahn/data/shape_dataset/data/brain_lightsheet'
from_index=88
start_index=0
to_index=215
num_of_slices=20
tickness = 0.02


from PIL import Image
im = Image.open('/data/jhahn/data/brain_lightsheet/0408/0000.tif')
from tifffile import imread
image = imread('/data/jhahn/data/brain_lightsheet/0408/0001.tif')
image.shape
import tifffile

def stack_tiffs(file_pattern):
    """Stacks multiple TIFF files into a 3D numpy array."""
    file_list = sorted(glob.glob(file_pattern)) # Sort files for consistent order
    if not file_list:
        raise FileNotFoundError(f"No files found matching pattern: {file_pattern}")
    
    stack = []
    for file_path in file_list:
        with tifffile.TiffFile(file_path) as tif:
            stack.append(tif.asarray())
    
    return np.stack(stack, axis=0)

# Example usage
file_pattern = '/data/jhahn/data/brain_lightsheet/0408/*.tif'
try:
    image_stack = stack_tiffs(file_pattern)
    print(f"Image stack shape: {image_stack.shape}")
except FileNotFoundError as e:
    print(e)



FileNotFoundError: [Errno 2] No such file or directory: '/data/jhahn/data/brain_lightsheet/0408/0000.tif'

In [11]:
import slice_util
import trimesh
import numpy as np
import os
obj_file_list = []
glb_dir = "/data/jhahn/data/shape_dataset/data/brain_lightsheet/sliced_on_0_1_0_0.001_50_105_700/fractured_0/"
files = os.listdir(glb_dir)
for f in files:
    _glb = trimesh.load(glb_dir+f)
    _glb.export('output/'+f.replace(".glb",".obj"))
    obj_file_list.append('output/'+f.replace(".glb",".obj"))
slice_util.combine_obj_files(obj_file_list,'test.obj')




[]

In [ ]:
from tifffile import imread
image = imread('/data/jhahn/data/brain_lightsheet/mask_stack.tif')
image.shape


In [5]:
import trimesh
_glb = trimesh.load("/data/jhahn/data/shape_dataset/data/brain_lightsheet/mask_0.0_0.0_0.1_gap_1_0.001_0.007/fractured_0/0100.glb")
_obj = trimesh.load("/data/jhahn/data/shape_dataset/data/mouse_brain_50mm/50_tickness_20_sllices_test/fractured_0/piece_0.obj")

In [9]:
# 1. 씬에 있는 모든 지오메트리의 정점(vertex) 배열들을 리스트로 수집합니다.
all_vertices = [geom.vertices for geom in _glb.geometry.values()]

# 2. NumPy의 vstack을 사용해 모든 정점 배열을 하나의 큰 배열로 합칩니다.
combined_vertices = np.vstack(all_vertices)

# 3. 합쳐진 정점들로 PointCloud 객체를 생성합니다.
_glb_w = trimesh.PointCloud(vertices=combined_vertices)
_glb_w

<trimesh.PointCloud(vertices.shape=(86608, 3))>

In [6]:
_obj

<trimesh.PointCloud(vertices.shape=(431, 3), name=`piece_0.obj`)>

In [1]:
dir = '/data/jhahn/data/shape_dataset/data/brain_lightsheet'
import random
import numpy as np

import os
import shutil
_dir_list = os.listdir(dir)
_dir_list = [w for w in _dir_list if w.endswith('_700')]
_dir_list

# 그룹 비율을 설정합니다 (총합이 1이 되어야 합니다).
group_ratios = [0.7, 0.2, 0.1] 

# --- 여기서부터는 수정할 필요가 없습니다. ---

# 폴더 목록을 무작위로 섞습니다.
random.shuffle(_dir_list)

# 총 폴더 수를 계산합니다.
total_folders = len(_dir_list)

# 각 그룹의 크기를 계산하고, 리스트를 나눌 인덱스를 결정합니다.
split_indices = np.cumsum([int(total_folders * r) for r in group_ratios[:-1]])

# 계산된 인덱스를 기준으로 리스트를 3개의 그룹으로 나눕니다.
groups = np.split(_dir_list, split_indices)
# 결과를 출력합니다.
group1 = list(groups[0])
group2 = list(groups[1])
group3 = list(groups[2])

print(f"📂 그룹 1 ({len(group1)}개, {group_ratios[0]*100:.0f}%):", group1)
print(f"📂 그룹 2 ({len(group2)}개, {group_ratios[1]*100:.0f}%):", group2)
print(f"📂 그룹 3 ({len(group3)}개, {group_ratios[2]*100:.0f}%):", group3)

for data_type, data in [('train',group1),('test',group2),('val',group3)]:
    with open(f'/data/jhahn/data/shape_dataset/data/brain_lightsheet.{data_type}.txt',"w") as output_file:
                
        for f in sorted(data):
            print(f)
            #if len(f.split("_")) ==2:
            #    continue
            #if f.endswith(data_type):
            output_file.write(f'brain_lightsheet/{f}\n')
                #print(f'brain_lightsheet/{f}')
            '''
            for qq in range(5,100):
                if f.endswith("val") and qq >= 5:
                    
                    if os.path.exists(f'{dir}/{f}/fractured_{qq}'):
                        shutil.rmtree(f'{dir}/{f}/fractured_{qq}')
                elif f.endswith("train") and qq >= 50:
                    if os.path.exists(f'{dir}/{f}/fractured_{qq}'):
                        shutil.rmtree(f'{dir}/{f}/fractured_{qq}')
            '''
    #
#shutil.rmtree("my_non_empty_directory")

📂 그룹 1 (9604개, 70%): ['sliced_on_0_1_1_0.001_35_119_700', 'sliced_on_1_1_0_0.001_45_256_700', 'sliced_on_1_0_1_0.001_50_007_700', 'sliced_on_1_0_1_0.001_45_151_700', 'sliced_on_1_1_0_0.001_40_035_700', 'sliced_on_1_1_0_0.001_50_186_700', 'sliced_on_0_0_1_0.001_35_210_700', 'sliced_on_1_1_1_0.001_20_055_700', 'sliced_on_0_1_0_0.001_20_008_700', 'sliced_on_1_0_0_0.001_40_235_700', 'sliced_on_1_1_1_0.001_50_045_700', 'sliced_on_0_1_1_0.001_45_259_700', 'sliced_on_0_1_1_0.001_40_283_700', 'sliced_on_1_0_0_0.001_35_067_700', 'sliced_on_1_0_0_0.001_45_041_700', 'sliced_on_0_1_1_0.001_40_251_700', 'sliced_on_1_0_0_0.001_50_137_700', 'sliced_on_1.0_1.0_0.0_0.001_20_051_700', 'sliced_on_1_1_1_0.001_40_116_700', 'sliced_on_0_0_1_0.001_20_268_700', 'sliced_on_1_0_0_0.001_30_117_700', 'sliced_on_1_1_1_0.001_35_119_700', 'sliced_on_1.0_1.0_0.0_0.001_45_014_700', 'sliced_on_1.0_1.0_0.0_0.001_35_011_700', 'sliced_on_1_1_0_0.001_40_330_700', 'sliced_on_1_0_0_0.001_50_030_700', 'sliced_on_0_1_1_0.001_3